In [14]:
import torch
import pandas as pd
import numpy as np
from rdkit import Chem
import torch_geometric.data as pyg_data
from sklearn.model_selection import train_test_split
import torch_geometric.nn as pyg_nn
import torch.nn as nn
import torch 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch_geometric.data as pyg_data

In [15]:
data = pd.read_csv("../data/raw/odor.csv")

In [16]:
data.head()

,nonStereoSMILES,descriptors,alcoholic,aldehydic,alliaceous,almond,amber,animal,anisic,apple,...,tropical,vanilla,vegetable,vetiver,violet,warm,waxy,weedy,winey,woody
0,CC(O)CN,fishy,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CCC(=O)C(=O)O,fatty;lactonic;sweet;caramellic;creamy,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,O=C(O)CCc1ccccc1,rose;floral;fatty;sweet;musk;cinnamon;balsamic,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,OCc1ccc(O)cc1,medicinal;phenolic;fruity;nutty;bitter;sweet;a...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,O=Cc1ccc(O)cc1,phenolic;woody;nutty;vanilla;hay;metallic;swee...,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [17]:
data = data.drop(['descriptors'], axis=1)

In [18]:
round(100*(data.isnull().sum()/len(data.index)), 2)

nonStereoSMILES    0.0
alcoholic          0.0
aldehydic          0.0
alliaceous         0.0
almond             0.0
                  ... 
warm               0.0
waxy               0.0
weedy              0.0
winey              0.0
woody              0.0
Length: 139, dtype: float64

In [19]:
class GNNModel(nn.Module):
    """
    A Graph Neural Network (GNN) model for node classification.

    This model consists of two graph convolutional layers, followed by a global mean pooling layer and a fully connected layer.

    Attributes:
    conv1 (pyg_nn.GraphConv): The first graph convolutional layer.
    conv2 (pyg_nn.GraphConv): The second graph convolutional layer.
    pool (pyg_nn.global_mean_pool): The global mean pooling layer.
    fc1 (nn.Linear): The fully connected layer.
    """
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = pyg_nn.GraphConv(3, 128)
        self.conv2 = pyg_nn.GraphConv(128, 128)
        self.pool = pyg_nn.global_mean_pool
        self.fc1 = nn.Linear(128, 138)

    def forward(self, data):
        """
        The forward pass of the model.

        Parameters:
        data (pyg_data.Data): The input data, containing node features, edge indices, and batch indices.

        Returns:
        torch.Tensor: The output of the model, representing the predicted node labels.
        """
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))
        x = self.pool(x, batch)
        x = self.fc1(x)
        return torch.sigmoid(x)

In [20]:
def load_model(path):
    """
    Loads a pre-trained GNN model from the specified file path.

    Args:
        model_path (str): The file path to the model's state dictionary.

    Returns:
        GNNModel: An instance of the GNNModel class with the loaded parameters, set to evaluation mode.
    """
    model = GNNModel()
    model.load_state_dict(torch.load(path))  
    model.eval()  
    return model

model = load_model("../models/model.pt")

C:\Users\Stefano\AppData\Local\Temp\ipykernel_11896\1857281581.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


In [21]:
def create_data_list(smiles_list, labels):
    """
    Creates a data list for PyTorch Geometric from SMILES strings and labels.

    Parameters:
    smiles_list: List of SMILES strings.
    labels: DataFrame of labels.

    Returns:
    list: List of data in PyTorch Geometric format.
    """
    data_list = []
    max_nodes = 0
    for i, smiles in enumerate(smiles_list):
        molecule = Chem.MolFromSmiles(smiles)
        atom_features = []
        for atom in molecule.GetAtoms():
            features = []
            features.append(atom.GetAtomicNum())
            features.append(atom.GetDegree())
            features.append(atom.GetTotalValence())
            atom_features.append(features)

        edge_index = []
        for bond in molecule.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_index.append([i, j])
            edge_index.append([j, i])
        edge_index = torch.tensor(edge_index, dtype=torch.long).T

        atom_features = np.array(atom_features, dtype=np.float32)


        max_nodes = max(max_nodes, len(atom_features))

        data = pyg_data.Data(x=torch.tensor(atom_features),
                             edge_index=edge_index,
                             y=torch.tensor(labels.iloc[i].values.astype(np.float32)))
        data_list.append(data)

    for data in data_list:
        num_nodes = data.x.size(0)
        if num_nodes < max_nodes:
            padding = torch.zeros((max_nodes - num_nodes, data.x.size(1)))
            data.x = torch.cat((data.x, padding), dim=0)

    return data_list


In [22]:
train_smiles, test_smiles, train_labels, test_labels = train_test_split(data["nonStereoSMILES"], data.iloc[:, 1:], test_size=0.2, random_state=42)

In [23]:
def evaluate_model(model, data_loader):
    """
    Evaluates the performance of a machine learning model on a dataset.

    This function sets the model to evaluation mode and calculates performance metrics
    such as accuracy, weighted precision, weighted recall, and weighted F1 score
    based on the predictions made by the model and the true labels.

    Args:
        model (torch.nn.Module): The PyTorch model to be evaluated.
        data_loader (torch.utils.data.DataLoader): A DataLoader that provides the input data
            and corresponding labels for evaluation.

    Returns:
        dict: A dictionary containing the evaluation metrics:
            - 'accuracy' (float): The accuracy of the model.
            - 'precision' (float): The weighted precision of the model.
            - 'recall' (float): The weighted recall of the model.
            - 'f1_score' (float): The weighted F1 score of the model.

    Raises:
        ValueError: If the shapes of the true labels and predictions do not match.

    Notes:
        - The function disables gradient calculation to optimize performance during evaluation.
        - Predictions are binarized using a threshold of 0.5.
        - Metrics are calculated using sklearn functions to ensure accurate evaluation.
    """
    model.eval()  # Set the model to evaluation mode
    all_labels = []
    all_predictions = []

    with torch.no_grad():  # Disable gradient calculation
        for data in data_loader:
            output = model(data)  # Forward pass
            predictions = (output > 0.5).float()  # Binarize predictions
            
            
            all_labels.append(data.y.cpu().numpy())
            all_predictions.append(predictions.cpu().numpy())

    # Concatenate all labels and predictions
    all_labels = np.concatenate(all_labels)
    all_predictions = np.concatenate(all_predictions)


    # Reshape predictions if necessary
    if all_predictions.ndim > 1:
        all_predictions = all_predictions.reshape(-1)  # Flatten to 1D if needed

    # Ensure that all_labels and all_predictions have the same shape
    if all_labels.shape != all_predictions.shape:
        raise ValueError(f"Shape mismatch: {all_labels.shape} vs {all_predictions.shape}")

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

In [24]:
test_data_list = create_data_list(test_smiles, test_labels)

test_loader = pyg_data.DataLoader(test_data_list, batch_size=32, shuffle=False)

metrics = evaluate_model(model, test_loader)

print("Métricas de evaluación:")
print(f"Exactitud: {metrics['accuracy']:.4f}")
print(f"Precisión: {metrics['precision']:.4f}")
print(f"Recuperación: {metrics['recall']:.4f}")
print(f"Puntuación F1: {metrics['f1_score']:.4f}")

c:\Users\Stefano\anaconda3\envs\GNNOdor\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Métricas de evaluación:
Exactitud: 0.9327
Precisión: 0.9290
Recuperación: 0.9327
Puntuación F1: 0.9308
